### Import the packages

In [1]:
import numpy as np
import xarray as xr

### Test our text files that we wrote later in this same file

In [2]:
# original polynya locations used in submission 1
pol_locs = np.fromfile("/scratch/project_2000789/muramarg/floats_WAOM/text_files/pollocs_dsw.txt",sep=",")
pol_locs = pol_locs.astype(int)

In [3]:
# select from polynyaidx_3_26.txt that forms DSW - this is from our original analysis, should ideally match
pol_locs1 = np.fromfile("/scratch/project_2000789/muramarg/floats_WAOM/text_files/polynyaidx_3_26.txt",sep=",")
pol_locs1 = pol_locs1.astype(int)

In [4]:
np.intersect1d(pol_locs,pol_locs1).tofile("/scratch/project_2000789/muramarg/floats_WAOM/text_files/trythis_polidx.txt",sep=",")

In [5]:
pol_locs1.shape

(7599,)

In [6]:
# select wmt from 3_26 files
wmt = np.fromfile("/scratch/project_2000789/muramarg/floats_WAOM/text_files/wmt_3_26.txt",sep=",")
wmt = wmt.reshape((35040,-1))

In [7]:
columns_with_zero = np.any(wmt == 3, axis=0)
indices_with_zero = np.where(columns_with_zero)[0]

print(pol_locs1[indices_with_zero].shape)

(5991,)


In [8]:
# select from polynyaidx_3_26.txt that forms DSW - this is from our original analysis, should ideally match
pol_locs2 = np.fromfile("/scratch/project_2000789/muramarg/floats_WAOM/text_files/polynyaidx_3_27_limited.txt",sep=",")
pol_locs2 = pol_locs2.astype(int)

In [9]:
# np.intersect1d(pol_locs2,pol_locs1[indices_with_zero]).shape
np.intersect1d(pol_locs2,pol_locs).shape

(1745,)

In [58]:
# set these to a file to show an example for clustering
pol_locs1[indices_with_zero].tofile("/scratch/project_2000789/muramarg/floats_WAOM/text_files/polynya_-120_1.3e-5.txt",sep=",")

In [59]:
# set these to a file to show an example for clustering
np.intersect1d(pol_locs2,pol_locs1[indices_with_zero]).tofile("/scratch/project_2000789/muramarg/floats_WAOM/text_files/polynya_-125_1.4e-5.txt",sep=",")

### Restrict where we set polynyas

In [4]:
# this is based on the files polynya_seasons and pollocs_dsw
pol_locs = np.fromfile("/scratch/project_2000789/muramarg/floats_WAOM/text_files/polynya_locs.txt",sep=",")
pol_locs.shape

(5328,)

In [24]:
points = np.fromfile("/scratch/project_2000789/muramarg/floats_WAOM/text_files/points.txt",sep=",")
points = points.reshape(391,2)
points.shape

(391, 2)

In [25]:
xgrid = np.fromfile("/scratch/project_2000789/muramarg/floats_WAOM/text_files/xgrid_dsw.txt",sep=",")
ygrid = np.fromfile("/scratch/project_2000789/muramarg/floats_WAOM/text_files/ygrid_dsw.txt",sep=",")

xgrid = xgrid.reshape((35040,-1))
ygrid = ygrid.reshape((35040,-1))

In [26]:
# collapse the points based on this
firstx = xgrid[0]
firsty = ygrid[0]

In [27]:
points = np.stack((firstx,firsty)).T

In [28]:
unique_set = set(map(tuple, points))

# Convert the set back to a NumPy array
unique_array = np.array(list(unique_set))

# Resize the array if needed
# For example, if you want to resize it to shape (n, 2), where n is the number of unique points
# You might need to adjust n based on the actual number of unique points
n = len(unique_array)
unique_array = unique_array[:n]

print(unique_array.shape)
# unique_array is now the list of release points we use in the simulation

(328, 2)


### Let's find the time steps where there is a polynya

In [29]:
def sh_points(ds,thres,month,xr,yr):
    """
    inputs: the dataset
    thres: the threshold value for flux
    month: the index of desired month
    xr: range of x values to include
    yr: range of y values to include
    
    outputs: the x and y grid locations for the threshold values
    """
    # sh_arr = (ds.shflux.values)[month]
    sh_arr = np.mean(ds.shflux.values,axis=0)
    locs = np.where(sh_arr < thres)         # set values above the heat loss threshold to 0

    # filter by area on the map
    newx = np.array([])
    newy = np.array([])
    for i,j in zip(locs[1],locs[0]):
        if (i>xr[0] and i<xr[1]) and (j>yr[0] and j<yr[1]):
            newx = np.append(i,newx)
            newy = np.append(j,newy)
            
    return newx,newy

def ss_points(ds,thres,month,xr,yr):
    """
    inputs:
        ds: the dataset
        thres: the threshold value for flux
        month: the index of desired month
        xr: range of x values to include
        yr: range of y values to include
    
    outputs: 
        newx, newy: the x and y grid locations for the threshold values
    """
    
    #ss_arr = (ds.ssflux.values)[month]
    ss_arr = np.mean(ds.ssflux.values,axis=0)
    locs = np.where(ss_arr > thres)         # set values above the heat loss threshold to 0

    # filter by area on the map
    newx = np.array([])
    newy = np.array([])
    for i,j in zip(locs[1],locs[0]):
        if (i>xr[0] and i<xr[1]) and (j>yr[0] and j<yr[1]):
            newx = np.append(i,newx)
            newy = np.append(j,newy)
            
    return newx,newy

def mergepoints(x1,y1,x2,y2,h):
    """
    inputs:
        x1,y1: the x and y coordinates of two arrays
        x2,y2: the x and y coordinates of two arrays
    outputs:
        x,y: the combined arrays of two datapoints
    """
    # create the new set of points
    pts1 = np.empty((0,2))
    pts2 = np.empty((0,2))
    
    # read through each array, create points as tuples
    for i,j in zip(x1,y1):
        l = np.array([i,j])
        pts1 = np.append(np.array([l]),pts1,axis=0)
    for i,j in zip(x2,y2):
        l = np.array([i,j])
        pts2 = np.append(np.array([l]),pts2,axis=0)
        
    # union of the two sets
    set1 = set([tuple(x) for x in pts1])
    set2 = set([tuple(x) for x in pts2])
    pts = np.array([x for x in set1 and set2])
#     pts2 = np.array([x for x in set1 and set2])
#     for pt in pts2:
#         #print(pt)
#         pts = np.append(np.array([pt]),pts,axis=0)
    
    # separate into x and y arrays again
    x = np.array([])
    y = np.array([])
    for pt in pts:
        #print(pt[0],pt[1])
        thish = h[int(pt[1]),int(pt[0])]
        #print(thish)
        if thish<1000:
            x = np.append(pt[0],x)
            y = np.append(pt[1],y)
    
    return x,y
    
# xh,yh = sh_points(ds,-120,6,[450,600],[300,450])
# xs,ys = ss_points(ds,1.3e-5,6,[450,600],[300,450])
# plt.plot(xh,yh,'.',color="r",alpha=0.5,label='shflux points')
# plt.plot(xs,ys,'.',color="b",alpha=0.5,label='ssflux points')
# plt.title("sh and ss of points")
# plt.legend()

In [10]:
ds1  = xr.open_dataset("/scratch/project_2000789/muramarg/waom_total/output_WAOM_check/ocean_avg_0001.nc")
ds2  = xr.open_dataset("/scratch/project_2000789/muramarg/waom_total/output_WAOM_check/ocean_avg_0002.nc")
ds3  = xr.open_dataset("/scratch/project_2000789/muramarg/waom_total/output_WAOM_check/ocean_avg_0003.nc")
ds4  = xr.open_dataset("/scratch/project_2000789/muramarg/waom_total/output_WAOM_check/ocean_avg_0004.nc")
ds5  = xr.open_dataset("/scratch/project_2000789/muramarg/waom_total/output_WAOM_check/ocean_avg_0005.nc")
ds6  = xr.open_dataset("/scratch/project_2000789/muramarg/waom_total/output_WAOM_check/ocean_avg_0006.nc")
ds7  = xr.open_dataset("/scratch/project_2000789/muramarg/waom_total/output_WAOM_check/ocean_avg_0007.nc")
ds8  = xr.open_dataset("/scratch/project_2000789/muramarg/waom_total/output_WAOM_check/ocean_avg_0008.nc")
ds9  = xr.open_dataset("/scratch/project_2000789/muramarg/waom_total/output_WAOM_check/ocean_avg_0009.nc")
ds10 = xr.open_dataset("/scratch/project_2000789/muramarg/waom_total/output_WAOM_check/ocean_avg_0010.nc")
ds11 = xr.open_dataset("/scratch/project_2000789/muramarg/waom_total/output_WAOM_check/ocean_avg_0011.nc")
ds12 = xr.open_dataset("/scratch/project_2000789/muramarg/waom_total/output_WAOM_check/ocean_avg_0012.nc")

In [14]:
ds1.shflux

<xarray.DataArray 'shflux' (ocean_time: 4, eta_rho: 560, xi_rho: 630)>
[1411200 values with dtype=float32]
Coordinates:
    x_rho       (eta_rho, xi_rho) float64 ...
    y_rho       (eta_rho, xi_rho) float64 ...
  * ocean_time  (ocean_time) datetime64[ns] 2026-12-30T19:15:00 ... 2027-01-2...
Dimensions without coordinates: eta_rho, xi_rho
Attributes:
    long_name:       time-averaged surface net heat flux
    units:           watt meter-2
    negative_value:  upward flux, cooling
    positive_value:  downward flux, heating
    time:            ocean_time
    field:           surface heat flux, scalar, series

In [31]:
dg = xr.open_dataset("/scratch/project_2000789/muramarg/copied_files/waom10extend_grd.nc")

In [33]:
polynyas = np.zeros((12,104,2))

In [41]:
i = 0
for ds in ds1,ds2,ds3,ds4,ds5,ds6,ds7,ds8,ds9,ds10,ds11,ds12:
    #xh,yh = sh_points(ds,-120,6,[450,600],[300,450])
    #xs,ys = ss_points(ds,1.3e-5,6,[450,600],[300,450])
    xh,yh = sh_points(ds,-125,6,[450,600],[300,450])
    xs,ys = ss_points(ds,1.4e-5,6,[450,600],[300,450])
    
    x,y = mergepoints(xh,yh,xs,ys,dg.h.values)
    
    stack = np.stack((x,y)).T
    intersect = np.zeros(unique_array.shape)
    print(x.shape)
    #stackflat = print(stack.shape)
    #print(unique_array.shape)
    for p in range(len(stack)):
        if stack[p] in unique_array:
            intersect[p] = stack[p]
            
            
    non_zero_rows = ~np.all(intersect == 0, axis=1)

    # Filter out rows with all zeros
    arr_filtered = intersect[non_zero_rows]
    polynyas[i,:arr_filtered.shape[0],:]=arr_filtered
    i += 1

(0,)
(0,)
(7,)
(63,)
(16,)
(78,)
(75,)
(72,)
(41,)
(2,)
(0,)
(0,)


In [42]:
# now we can use arr_filtered to create a new list of indices in the array
df = xr.open_dataset("/scratch/project_2000789/muramarg/waom_total/output_WAOM_check/ocean_flt.nc")

In [43]:
xgrid = df.Xgrid.values
ygrid = df.Ygrid.values

In [44]:
first_non_nan_indices = []
for col in range(xgrid.shape[1]):
    # Find indices of non-NaN values in the column
    non_nan_indices = np.where(~np.isnan(xgrid[:, col]))[0]
    if len(non_nan_indices) > 0:
        # Select the first non-NaN value index
        first_non_nan_indices.append(non_nan_indices[0])

# Now, first_non_nan_indices contains the index of the first non-NaN value in each column
# Use these indices to extract the corresponding values
first_non_nan_values = xgrid[first_non_nan_indices, np.arange(xgrid.shape[1])]

print(first_non_nan_values)

[471. 471. 471. ... 597. 597. 597.]


In [45]:
first_non_nan_indices = []
for col in range(ygrid.shape[1]):
    # Find indices of non-NaN values in the column
    non_nan_indices = np.where(~np.isnan(ygrid[:, col]))[0]
    if len(non_nan_indices) > 0:
        # Select the first non-NaN value index
        first_non_nan_indices.append(non_nan_indices[0])

# Now, first_non_nan_indices contains the index of the first non-NaN value in each column
# Use these indices to extract the corresponding values
first_non_nan_valuesy = ygrid[first_non_nan_indices, np.arange(ygrid.shape[1])]

print(first_non_nan_valuesy)

[369. 369. 369. ... 428. 428. 428.]


In [46]:
stack_flt = np.stack((first_non_nan_values,first_non_nan_valuesy)).T
stack_flt.shape

(20332, 2)

In [47]:
# each release point in stack_flt should be released (52 for each)
stack_flt  # 391 points, 52 for each point

array([[471., 369.],
       [471., 369.],
       [471., 369.],
       ...,
       [597., 428.],
       [597., 428.],
       [597., 428.]], dtype=float32)

In [48]:
point_tuples = [tuple(row) for row in stack_flt]

# Use set() to find unique tuples
unique_points = set(point_tuples)

# Get the number of unique points
num_unique_points = len(unique_points)

print(num_unique_points)

391


In [49]:
# 4 4 5 4 4 4 5 4 5 4 5 

# for i in stack_flt
stupid_months = np.array([0,0,0,0,1,1,1,1,2,2,2,2,2,3,3,3,3,4,4,4,4,5,5,5,5,6,6,6,6,6,7,7,7,7,8,8,8,8,9,9,9,9,9,10,10,10,10,11,11,11,11,11])

In [50]:
months = np.zeros((20332))
pol_idx = np.array([],dtype=int)
firsti = 0

for point in range(len(stack_flt)):
    # get the points from the current polynya
    mym = stupid_months[firsti]
    mym = polynyas[mym]
    
    # see if point is in mym
    if stack_flt[point] in mym:
        pol_idx = np.append(pol_idx,point)
    
    firsti += 1
    if firsti == 51:
        firsti = 0

In [51]:
pol_idx.tofile("/scratch/project_2000789/muramarg/floats_WAOM/text_files/polynyaidx_3_27_limited.txt",sep=",")